<a href="https://colab.research.google.com/github/anshmehta337/-mnist-digit-classifier/blob/main/work.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

In [11]:
!pip install optuna

In [12]:
import optuna

In [13]:
#lets load our dataset too
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

trainset=torchvision.datasets.MNIST(root='./data',train=True,download=True,transform=transforms.ToTensor())
testset=torchvision.datasets.MNIST(root='./data',train=False,download=True,transform=transforms.ToTensor())

In [14]:
#lets load them
trainloader=torch.utils.data.DataLoader(trainset,batch_size=64,shuffle=True)
testloader=torch.utils.data.DataLoader(testset,batch_size=64,shuffle=False)

In [15]:

# Define model
class Net(nn.Module):
    def __init__(self, hidden_size, drop):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28 * 28, hidden_size)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(drop)
        self.fc2 = nn.Linear(hidden_size, 10)

    def forward(self, x):
        x = x.view(-1, 28 * 28)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# Define objective
def obj(trial):
    hidd = trial.suggest_int('hidden_size', 127, 514)
    dr = trial.suggest_float('dr', 0.0, 0.2)
    lr = trial.suggest_float('lr', 1e-4, 1e-1, log=True)
    opt_name = trial.suggest_categorical('optimizer', ['Adam', 'SGD'])

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    mod = Net(hidd, dr).to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = getattr(optim, opt_name)(mod.parameters(), lr=lr)

    # Training
    mod.train()
    for epoch in range(5):
        for inputs, labels in trainloader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = mod(inputs)
            loss = loss_fn(outputs, labels)
            loss.backward()
            optimizer.step()

    # Evaluation
    mod.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for inputs, labels in testloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = mod(inputs)
            _, pred = outputs.max(1)
            correct += (pred == labels).sum().item()
            total += labels.size(0)

    accuracy = correct / total
    return 1.0 - accuracy  # minimize error

# Run Optuna
study = optuna.create_study(direction='minimize')
study.optimize(obj, n_trials=30)

# Results
print("Best trial:")
print("  Accuracy: {:.4f}".format(1 - study.best_value))
print("  Params: ", study.best_params)


[I 2025-08-01 18:39:06,991] A new study created in memory with name: no-name-63a8490e-2cc7-473d-ac62-3929085c5199
[I 2025-08-01 18:39:51,724] Trial 0 finished with value: 0.10840000000000005 and parameters: {'hidden_size': 506, 'dr': 0.19056971033624284, 'lr': 0.05159235201410254, 'optimizer': 'Adam'}. Best is trial 0 with value: 0.10840000000000005.
[I 2025-08-01 18:40:31,689] Trial 1 finished with value: 0.24970000000000003 and parameters: {'hidden_size': 182, 'dr': 0.044096013519145584, 'lr': 0.0005252381632125263, 'optimizer': 'SGD'}. Best is trial 0 with value: 0.10840000000000005.
[I 2025-08-01 18:41:12,500] Trial 2 finished with value: 0.12490000000000001 and parameters: {'hidden_size': 140, 'dr': 0.11428353884118331, 'lr': 0.05780458842697311, 'optimizer': 'Adam'}. Best is trial 0 with value: 0.10840000000000005.
[I 2025-08-01 18:41:53,907] Trial 3 finished with value: 0.01970000000000005 and parameters: {'hidden_size': 465, 'dr': 0.11131041333323029, 'lr': 0.000643328683232597

Best trial:
  Accuracy: 0.9812
  Params:  {'hidden_size': 393, 'dr': 0.1532285171973303, 'lr': 0.0008924258557927509, 'optimizer': 'Adam'}
